# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Program: Computer Systems Engineering  </center>
---
### <center> **Spring 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: 31/03/2025

**Team Name**: Par De Dos

**Professor**: Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ParDeDosLogsStreaming") \
    .master("spark://368ad5a83fd7:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/03 19:57:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
log_dir="/home/jovyan/notebooks/data/structured_streaming_log_files/"

logs_lines = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 5) \
                .load(log_dir)

In [4]:
from pyspark.sql.functions import split, col

logs_df = logs_lines.select(split(col("value"), r"\s*\|\s*").alias("log_array"))

logs_df = logs_df.withColumn("date_time", col("log_array")[0])
logs_df = logs_df.withColumn("log_level", col("log_array")[1])
logs_df = logs_df.withColumn("message", col("log_array")[2])
logs_df = logs_df.withColumn("node", col("log_array")[3])


logs_df.printSchema()

root
 |-- log_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date_time: string (nullable = true)
 |-- log_level: string (nullable = true)
 |-- message: string (nullable = true)
 |-- node: string (nullable = true)



In [5]:
logs_df = logs_df.filter(logs_df.log_level == "ERROR")

In [6]:
query = logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='5 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(30)

25/04/03 19:58:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5611a9dc-388e-41d2-9169-463df7014612. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/03 19:58:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|log_array                                                                       |date_time          |log_level|message                            |node         |
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|[2025-04-03 19:55:10, ERROR, Unauthorized login attempt detected, server-node-1]|2025-04-03 19:55:10|ERROR    |Unauthorized login attempt detected|server-node-1|
|[2025-04-03 19:55:10, ERROR, Imminent Explosion, server-node-2]                 |2025-04-03 19:55:10|ERROR    |Imminent Explosion                 |server-node-2|
|[2025-04-03 19:55:10, ERROR, Unauthorized login attempt detected, server-node-2]|2025-0

25/04/03 19:58:07 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 5778 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|log_array                                                                       |date_time          |log_level|message                            |node         |
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|[2025-04-03 19:58:04, ERROR, Unauthorized login attempt detected, server-node-1]|2025-04-03 19:58:04|ERROR    |Unauthorized login attempt detected|server-node-1|
|[2025-04-03 19:58:04, ERROR, Imminent Explosion, server-node-1]                 |2025-04-03 19:58:04|ERROR    |Imminent Explosion                 |server-node-1|
|[2025-04-03 19:58:04, ERROR, Database connection lost, server-node-1]           |2025-0

False

-------------------------------------------
Batch: 7
-------------------------------------------
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|log_array                                                                       |date_time          |log_level|message                            |node         |
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|[2025-04-03 19:58:36, ERROR, Unauthorized login attempt detected, server-node-3]|2025-04-03 19:58:36|ERROR    |Unauthorized login attempt detected|server-node-3|
|[2025-04-03 19:58:36, ERROR, Imminent Explosion, server-node-2]                 |2025-04-03 19:58:36|ERROR    |Imminent Explosion                 |server-node-2|
|[2025-04-03 19:58:36, ERROR, Imminent Explosion, server-node-1]                 |2025-0

In [8]:
query.stop()

In [7]:
sc.stop()